In [ ]:
# MCP Server & Client Implementation Using Fastmcp
# Architecture
# LLM Host (MCP Client) <—— JSON-RPC ——> fastmcp Server <——> CSV Dataset

# We'll:

# Launch a server exposing two tools (summarize, query).
# Use an MCP Client to discover and call those tools.
# Simulate a host (agent) deciding which tool to call.
# Start the MCP Server
# In a separate terminal, run:

# python mcp_server_fastmcp.py
# This starts an MCP-compliant server exposing summarize and query tools.

In [ ]:
%pip install fastmcp pandas openai 
# uvicorn --upgrade --no-cache-dir


In [ ]:
# Connect via MCP Client
# Once the server is running, connect using the MCP client to list tools and invoke them.


from fastmcp import Client
from fastmcp.client import PythonStdioTransport
import asyncio

async def main():
    # Create a client using the PythonStdioTransport
    # Create a loop to run the client

    transport = PythonStdioTransport("mcp_server_fastmcp.py")
    
    # client = Client(transport)

    async with Client(transport) as client:
        tools = await client.list_tools()
        print("Tools:", [t.name for t in tools])
        result = await client.call_tool("summarize", {"dataset": "path/to/dataset.csv"})
        print("Summary:", result)
 

In [ ]:
# Simulate a Simple Host Agent
# Let's simulate a rule-based 'AI agent' that decides whether to use summarize or query based on user text.

def decide_tool(text: str):
    text = text.lower()
    if "summarize" in text or "overview" in text:
        return "summarize", {}
    if "west" in text:
        return "query", {"expr": "region == 'West' and sales > 1500"}
    return "summarize", {}

async def run_agent(user_input, client):
    tool, params = decide_tool(user_input) #"summarize", {}
    #tool = "summarize"
    #params = {}
    print(f"Agent decided to use '{tool}'")

    # API for fastmcp 2.12.5
    result = await client.call_tool(tool, params)
    # result = await client.call_tool("query", params)

    print("Result:", result, "\n")

In [ ]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)

In [ ]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)
    await run_agent("Show West region sales > 1500", client)

In [ ]:

# Architecture Recap
# +--------------------+      JSON-RPC (MCP)      +--------------------+
# |  Host / AI Agent   |  <-------------------->  | fastmcp MCP Server |
# | (LLM or Router)    |                        | (summarize, query)  |
# +--------------------+                         +--------------------+